### SparkSession
* Encapsulates multiple contexts
* One multiple sessions will share one of each context

In [9]:
from pyspark.sql import SparkSession
spark  = SparkSession.builder.master("local").getOrCreate()
sc = spark.sparkContext

In [10]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

### From RDD
Just add a schema

In [15]:
sample_list = [('Mona',20), ('Jennifer',34), ('John',20), ('Jim',26)]
rdd = sc.parallelize(sample_list)
names_df = spark.createDataFrame(rdd, schema=['Name', 'Age'])
names_df.head(3)

[Row(Name='Mona', Age=20),
 Row(Name='Jennifer', Age=34),
 Row(Name='John', Age=20)]

### From File
Interpret already pre-structured data

In [16]:
csv_df = spark.read.csv('dataframe.csv', header=True, inferSchema=True)
csv_df.head(2)

[Row(A=1, B='a', C='x'), Row(A=2, B='b', C='y')]

### Transformations and Actions
* Very similar to pandas
* Transform and then show

In [53]:
df = spark.read.csv('sql-practice.csv', header=True, inferSchema=True)
df.printSchema()

root
 |-- policyID: integer (nullable = true)
 |-- statecode: string (nullable = true)
 |-- county: string (nullable = true)
 |-- eq_site_limit: integer (nullable = true)
 |-- hu_site_limit: double (nullable = true)
 |-- fl_site_limit: integer (nullable = true)
 |-- fr_site_limit: double (nullable = true)
 |-- tiv_2011: double (nullable = true)
 |-- tiv_2012: double (nullable = true)
 |-- eq_site_deductible: integer (nullable = true)
 |-- hu_site_deductible: double (nullable = true)
 |-- fl_site_deductible: integer (nullable = true)
 |-- fr_site_deductible: integer (nullable = true)
 |-- point_latitude: double (nullable = true)
 |-- point_longitude: double (nullable = true)
 |-- line: string (nullable = true)
 |-- construction: string (nullable = true)
 |-- point_granularity: integer (nullable = true)



In [54]:
df.withColumnRenamed('policyID','ID').printSchema()

root
 |-- ID: integer (nullable = true)
 |-- statecode: string (nullable = true)
 |-- county: string (nullable = true)
 |-- eq_site_limit: integer (nullable = true)
 |-- hu_site_limit: double (nullable = true)
 |-- fl_site_limit: integer (nullable = true)
 |-- fr_site_limit: double (nullable = true)
 |-- tiv_2011: double (nullable = true)
 |-- tiv_2012: double (nullable = true)
 |-- eq_site_deductible: integer (nullable = true)
 |-- hu_site_deductible: double (nullable = true)
 |-- fl_site_deductible: integer (nullable = true)
 |-- fr_site_deductible: integer (nullable = true)
 |-- point_latitude: double (nullable = true)
 |-- point_longitude: double (nullable = true)
 |-- line: string (nullable = true)
 |-- construction: string (nullable = true)
 |-- point_granularity: integer (nullable = true)



In [55]:
# Select
df = df.select('policyID','statecode','county','eq_site_limit','hu_site_limit','fl_site_limit','fr_site_limit')
df.show()

+--------+---------+---------------+-------------+-------------+-------------+-------------+
|policyID|statecode|         county|eq_site_limit|hu_site_limit|fl_site_limit|fr_site_limit|
+--------+---------+---------------+-------------+-------------+-------------+-------------+
|  671392|       FL|    CLAY COUNTY|            0|      1.341E7|            0|          0.0|
|  772887|       FL|    CLAY COUNTY|            0|   1669113.93|            0|          0.0|
|  983122|       FL|    CLAY COUNTY|            0|    179562.23|            0|          0.0|
|  934215|       FL|    CLAY COUNTY|            0|    177744.16|            0|          0.0|
|  385951|       FL|    CLAY COUNTY|            0|     17757.58|            0|          0.0|
|  716332|       FL|    CLAY COUNTY|            0|    130129.87|            0|          0.0|
|  751262|       FL|    CLAY COUNTY|            0|     42854.77|            0|          0.0|
|  633663|       FL|    CLAY COUNTY|            0|       785.58|      

In [46]:
# Drop duplicates
df.select('statecode','county','eq_site_limit').drop_duplicates().show()

+---------+---------------+-------------+
|statecode|         county|eq_site_limit|
+---------+---------------+-------------+
|       FL|    CLAY COUNTY|            0|
|       FL|SUWANNEE COUNTY|            0|
|       FL|SUWANNEE COUNTY|       218475|
+---------+---------------+-------------+



In [41]:
# Filter
df.filter(df.policyID < 700000).show()

+--------+---------+---------------+-------------+-------------+-------------+-------------+
|policyID|statecode|         county|eq_site_limit|hu_site_limit|fl_site_limit|fr_site_limit|
+--------+---------+---------------+-------------+-------------+-------------+-------------+
|  671392|       FL|    CLAY COUNTY|            0|      1.341E7|            0|          0.0|
|  385951|       FL|    CLAY COUNTY|            0|     17757.58|            0|          0.0|
|  633663|       FL|    CLAY COUNTY|            0|       785.58|            0|          0.0|
|  105851|       FL|    CLAY COUNTY|            0|    170361.91|            0|          0.0|
|  352792|       FL|    CLAY COUNTY|            0|    366285.62|            0|          0.0|
|  294022|       FL|SUWANNEE COUNTY|            0|     96164.64|            0|          0.0|
|  410500|       FL|SUWANNEE COUNTY|            0|     11095.92|            0|          0.0|
|  524433|       FL|SUWANNEE COUNTY|       218475|     218475.0|      

In [43]:
# Group By
df.groupby('county').mean('hu_site_limit').show()

+---------------+------------------+
|         county|avg(hu_site_limit)|
+---------------+------------------+
|    CLAY COUNTY|1255265.5707692306|
|SUWANNEE COUNTY| 83745.54000000001|
+---------------+------------------+



In [45]:
# Order By
df.orderBy('policyID').show(5)

+--------+---------+---------------+-------------+-------------+-------------+-------------+
|policyID|statecode|         county|eq_site_limit|hu_site_limit|fl_site_limit|fr_site_limit|
+--------+---------+---------------+-------------+-------------+-------------+-------------+
|  105851|       FL|    CLAY COUNTY|            0|    170361.91|            0|          0.0|
|  294022|       FL|SUWANNEE COUNTY|            0|     96164.64|            0|          0.0|
|  352792|       FL|    CLAY COUNTY|            0|    366285.62|            0|          0.0|
|  385951|       FL|    CLAY COUNTY|            0|     17757.58|            0|          0.0|
|  410500|       FL|SUWANNEE COUNTY|            0|     11095.92|            0|          0.0|
+--------+---------+---------------+-------------+-------------+-------------+-------------+
only showing top 5 rows



### SQL

In [58]:
# Create "SQL table"
df.createOrReplaceTempView('table1')

In [60]:
# SQL Query to Dataframe
spark.sql('SELECT * FROM TABLE1').show()

+--------+---------+---------------+-------------+-------------+-------------+-------------+
|policyID|statecode|         county|eq_site_limit|hu_site_limit|fl_site_limit|fr_site_limit|
+--------+---------+---------------+-------------+-------------+-------------+-------------+
|  671392|       FL|    CLAY COUNTY|            0|      1.341E7|            0|          0.0|
|  772887|       FL|    CLAY COUNTY|            0|   1669113.93|            0|          0.0|
|  983122|       FL|    CLAY COUNTY|            0|    179562.23|            0|          0.0|
|  934215|       FL|    CLAY COUNTY|            0|    177744.16|            0|          0.0|
|  385951|       FL|    CLAY COUNTY|            0|     17757.58|            0|          0.0|
|  716332|       FL|    CLAY COUNTY|            0|    130129.87|            0|          0.0|
|  751262|       FL|    CLAY COUNTY|            0|     42854.77|            0|          0.0|
|  633663|       FL|    CLAY COUNTY|            0|       785.58|      